In [84]:
## Import necessary packages
import rdkit
from rdkit import Chem
import pandas as pd
import rdkit.Chem.rdMolDescriptors
import rdkit.Chem.rdmolfiles

In [85]:
## Function that assigns an atom ID to each atom in molecule
def mol_with_atom_index( mol ):
    atoms = mol.GetNumAtoms()
    for idx in range( atoms ):
        mol.GetAtomWithIdx( idx ).SetProp( 'molAtomMapNumber', str(mol.GetAtomWithIdx( idx ).GetIdx()))
    return mol

In [86]:
## Define the bent structural pattern and unwanted functional groups
patt = Chem.MolFromSmarts("[!R]~[R]~[R]~[!R]")
diSpatt = Chem.MolFromSmiles("SS")
imidinePatt = Chem.MolFromSmarts("[C;!R]=[NX2;!H1;!R]")
azolePatt = Chem.MolFromSmarts("[N;!R]=[N;!R]")
hydrazinePatt = Chem.MolFromSmarts("[N;!R]-[N;!R]")
michAcceptPatt = Chem.MolFromSmarts("[O;!R]=[C;!R][C;!R]=[C;!R]")
hydrazonePatt = Chem.MolFromSmarts("[n;R]-[O;!R]")
azolePatt2 = Chem.MolFromSmarts("[N]=[N;!R]")
hydrazinePatt2 = Chem.MolFromSmarts("[N]-[N;!R]")

In [87]:
## Create empty list to indicate if each entry has pattern match
hasCShape = []
## Create empty list to errored entries and molecules with unwanted functional groups
notEvenChecked = []
## Create empty list to store matches meeting amine/acid requirement
hasAmineAcid = []

## Create empty list to store matches with an amine
hasAmineMatch = []
## Create empty list to store/identify amine matches with double bond character
removedWithDBnitro = []
amineMatchesWithNeighbors = []
removedWithNearbyDB = []

## Create empty list to store matches with a carboxlic acid
hasAcidMatch = []

## Create empty list to store ZINC IDs of matches
origLibNum = []
listOfMols = []

In [89]:
## Initiate variable to store iteration/molecule number
loopNum = 0
## Create empty list to store database entries not recognized as molecules
erroredMols = []
## Specify import file and create output file
suppl = Chem.SDMolSupplier('chunk4_wash_final.sdf')
w = Chem.SDWriter('chunk4_matches_May8.sdf')

80600


In [90]:
## Iterate through input library to check each molecule for a bent substructure match
for mol in suppl:
    mol1 = mol
    loopNum = loopNum + 1
    
    ## Check if current entry is recognized as a molecule
    if(type(mol) != rdkit.Chem.rdchem.Mol):
        hasCShape.append(False)
        erroredMols.append(loopNum)
        notEvenChecked.append(True)
        continue
    
    ## Check molecule for unwanted functional groups
    diSmatches = mol.GetSubstructMatches(diSpatt)
    imidineMatches = mol.GetSubstructMatches(imidinePatt)
    azoleMatches = mol.GetSubstructMatches(azolePatt)
    hydrazineMatches = mol.GetSubstructMatches(hydrazinePatt)
    michMatches = mol.GetSubstructMatches(michAcceptPatt)
    hydrazoneMatches = mol.GetSubstructMatches(hydrazonePatt)
    azoleMatches2 = mol.GetSubstructMatches(azolePatt2)
    hydrazineMatches2 = mol.GetSubstructMatches(hydrazinePatt2)
    
    ## Check for bent pattern only in molecules not containing any unwanted functional groups
    if(len(diSmatches) >= 1 or len(imidineMatches) >= 1 or len(azoleMatches) >= 1 or len(hydrazineMatches) >= 1 or len(michMatches) >= 1 or len(hydrazoneMatches) >= 1 or len(azoleMatches2) >= 1 or len(hydrazineMatches2) >= 1):
        hasCShape.append(False)
        notEvenChecked.append(True)
    else:
        ## Save and index current molcule and its bonds
        mol_with_atom_index(mol1)
        bond_inds = {str(sorted((bond.GetBeginAtomIdx(),bond.GetEndAtomIdx()))):bond.GetIdx() for bond in mol1.GetBonds()}
    
        ## Search molecule for bent shape
        matches = mol1.GetSubstructMatches(patt)
        
        ## Create empty list to save substructure matches that meet further pattern criteria
        final_matches = []

        ## Define substructures for amine and carboxylic acid
        amine = Chem.MolFromSmarts("[NX3;H2,H1;!$(NC=O)]")
        carbacid = Chem.MolFromSmarts("[CX3](=O)[OH,O-]")
        chargedamine = Chem.MolFromSmarts("[NX4+;H3,H2;!$(NC=O)]")
        
        ## Specify molecule without bent shapes as non-match
        if len(matches) == 0:
            hasCShape.append(False)

        ## Check molecule with bent shapes for other pattern criteria
        else:
            for match in matches:
                ## Save bonds in substructure extending off ring
                bond_idx_match1 = bond_inds[str(sorted((match[0],match[1])))]
                bond_idx_match2 = bond_inds[str(sorted((match[2],match[3])))]

                ## Fragment molecule on bonds extending off ring
                fragmentedMol = Chem.FragmentOnBonds(mol1, (bond_idx_match1,bond_idx_match2))
                fragments = Chem.rdmolops.GetMolFrags(fragmentedMol, asMols = True)

                ## Save the two fragments that do not contain the ring of the substructure in a list and check if one of these fragments contains an amine or acid and no ring
                listOfFrags = []
                amineAcidFrag = None
                otherFrag = None
                fragContainingRing = None
                for frag in fragments:
                    fragIndices = []
                    for atom in frag.GetAtoms():
                        fragIndices.append(atom.GetAtomMapNum())
                    if set(fragIndices).intersection(match[1:3]):
                        fragContainingRing = frag
                        ringFragIndices = fragIndices
                    elif (frag.HasSubstructMatch(amine) or frag.HasSubstructMatch(carbacid) or frag.HasSubstructMatch(chargedamine)) and Chem.rdMolDescriptors.CalcNumRings(frag) == 0:
                        amineAcidFrag = frag
                        amineAcidFragIndices = fragIndices
                        amineAcidPatternLink = list(set(fragIndices).intersection(match))
                    else:
                        otherFrag = frag
                        otherFragIndices = fragIndices

                ## Check if one fragment contains an amine or acid and no ring while other fragment contains at least one aromatic ring
                if amineAcidFrag is not None and otherFrag is not None:
                    if Chem.rdMolDescriptors.CalcNumAromaticRings(otherFrag) >= 1:
                        hasAmineAcid.append(True)
                        
                        ## Initiate variable to store distance from amine/acid to ring
                        ## Pattern requires amine/acid to be no more than 2 atoms from ring
                        pathDistance = []
                        
                        ## Check if charged amine-containing match meets additional requirements
                        if amineAcidFrag.HasSubstructMatch(amine):
                            hasAmineMatch.append(True)
                            amineMatches = mol1.GetSubstructMatches(amine)
                            
                            ## Identify and store amine nitrogen
                            for a in amineMatches:
                                if set(amineAcidFragIndices).intersection(a):
                                    nitrogen = mol1.GetAtomWithIdx(int(a[0]))
                            
                            ## Store length of path from ring to amine
                            if amineAcidPatternLink[0] != nitrogen.GetIdx():
                                pathDistance = rdkit.Chem.rdmolops.GetShortestPath(mol1, amineAcidPatternLink[0], nitrogen.GetIdx())
                            
                            ## Exclude amine-containing match if amine nitrogen has double bond
                            doubleBondedN = []
                            nitrogenBonds = nitrogen.GetBonds()
                            for aBond in nitrogenBonds:
                                if aBond.GetBondType() != rdkit.Chem.rdchem.BondType.SINGLE:
                                    doubleBondedN.append(True)
                                    removedWithDBnitro.append(True)
                            
                            ## Exclude amine-containing match if amine nitrogen has double bond character
                            correctAtomNeighbs = []
                            nitrogenHasDBcharacter = []
                            nitrogenNeighbors = nitrogen.GetNeighbors()
                            for neighb in nitrogenNeighbors:
                                if neighb.GetAtomicNum() == 1 or neighb.GetAtomicNum() ==6:
                                    correctAtomNeighbs.append(True)
                                    if neighb.IsInRing() == False:
                                        for bo in neighb.GetBonds():
                                            if (bo.GetBondType() == rdkit.Chem.rdchem.BondType.DOUBLE) and (bo.GetEndAtom().GetAtomicNum() == 16 or bo.GetEndAtom().GetAtomicNum() == 8 or bo.GetEndAtom().GetAtomicNum() == 7):
                                                nitrogenHasDBcharacter.append(True)
                                                removedWithNearbyDB.append(True)
                            
                            ## Indicate match passes all pattern requirements
                            if len(pathDistance) <= 2 and (len(correctAtomNeighbs) == len(nitrogenNeighbors)) and len(nitrogenHasDBcharacter) == 0 and len(doubleBondedN) == 0:
                                amineMatchesWithNeighbors.append(True)
                                final_matches.append(True)
                        
                        ## Check if charged amine-containing match meets additional requirements
                        elif amineAcidFrag.HasSubstructMatch(chargedamine):
                            hasAmineMatch.append(True)
                            amineMatches = mol1.GetSubstructMatches(chargedamine)
                            
                            ## Identify and store amine nitrogen
                            for a in amineMatches:
                                if set(amineAcidFragIndices).intersection(a):
                                    nitrogen = mol1.GetAtomWithIdx(int(a[0]))
                            
                            ## Store length of path from ring to amine
                            if amineAcidPatternLink[0] != nitrogen.GetIdx():
                                pathDistance = rdkit.Chem.rdmolops.GetShortestPath(mol1, amineAcidPatternLink[0], nitrogen.GetIdx())
                            
                            ## Exclude amine-containing match if amine nitrogen has double bond
                            doubleBondedN = []
                            nitrogenBonds = nitrogen.GetBonds()
                            for aBond in nitrogenBonds:
                                if aBond.GetBondType() != rdkit.Chem.rdchem.BondType.SINGLE:
                                    doubleBondedN.append(True)
                                    removedWithDBnitro.append(True)
                            
                            ## Exclude amine-containing match if amine nitrogen has double bond character
                            correctAtomNeighbs = []
                            nitrogenHasDBcharacter = []
                            nitrogenNeighbors = nitrogen.GetNeighbors()
                            for neighb in nitrogenNeighbors:
                                if neighb.GetAtomicNum() == 1 or neighb.GetAtomicNum() ==6:
                                    correctAtomNeighbs.append(True)
                                    if neighb.IsInRing() == False:
                                        for bo in neighb.GetBonds():
                                            if (bo.GetBondType() == rdkit.Chem.rdchem.BondType.DOUBLE) and (bo.GetEndAtom().GetAtomicNum() == 16 or bo.GetEndAtom().GetAtomicNum() == 8 or bo.GetEndAtom().GetAtomicNum() == 7):
                                                nitrogenHasDBcharacter.append(True)
                                                removedWithNearbyDB.append(True)
                            
                            ## Indicate match passes all pattern requirements
                            if len(pathDistance) <= 2 and (len(correctAtomNeighbs) == len(nitrogenNeighbors)) and len(nitrogenHasDBcharacter) == 0 and len(doubleBondedN) == 0:
                                amineMatchesWithNeighbors.append(True)
                                final_matches.append(True)
                        
                        ## Check if carboxylic acid-containing match meets additional requirements
                        elif amineAcidFrag.HasSubstructMatch(carbacid):
                            hasAcidMatch.append(True)
                            acidMatches = mol1.GetSubstructMatches(carbacid)
                            
                            ## Identify and store carboxylic acid carbon
                            for a in acidMatches:
                                if set(amineAcidFragIndices).intersection(a):
                                    carbon = mol1.GetAtomWithIdx(int(a[0]))
                            
                            ## Store length of path from ring to carboxylic acid
                            if amineAcidPatternLink[0] != carbon.GetIdx():
                                pathDistance = rdkit.Chem.rdmolops.GetShortestPath(mol1, amineAcidPatternLink[0], carbon.GetIdx())
                            
                            ## Indicate match passes all pattern requirements
                            if len(pathDistance) <= 2:
                                final_matches.append(True)


            ## Indicate if molecule has passed bent pattern requirements by sorting into lists
            if len(final_matches) >= 1:
                hasCShape.append(True)
                origLibNum.append(loopNum)
                listOfMols.append(Chem.MolToSmiles(mol))
                w.write(mol)
            else:
                hasCShape.append(False)

In [94]:
## Exports SMILES and ID number of all ZINC bent pattern matches to csv file
df = pd.DataFrame()
df['SMILES'] = listOfMols
df['ZINC ID'] = origLibNum
df.to_csv('chunk4_matches_May8_SmilesAndKey.csv')